# 并发编程

## 启动与停止线程

In [1]:
import time

def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(1)

from threading import Thread
t = Thread(target=countdown, args=(10, ))

t.run()

if t.is_alive():
    print('Still running')
else:
    print('Complete!')

T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1
Complete!


## 判断线程是否已经启动

In [2]:
from threading import Thread, Event
import time

def countdown(n, start_evt):
    print('countdown starting...')
    start_evt.set()
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(1)

start_evt = Event()
print('Launching countdown...')
t = Thread(target=countdown, args=(10, start_evt))
t.run()
start_evt.wait()
print('countdown is running')

Launching countdown...
countdown starting...
T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1
countdown is running


## 进程间通信

In [3]:
from queue import Queue
from threading import Thread
import random

def producer(out_q):
    while True:
        # get some data
        data = random.randint(0, 10)
        out_q.put(data)

def consumer(in_q):
    while True:
        data = in_q.get()
        # process data

# q = Queue()
# t1 = Thread(target=consumer, args=(q,))
# t2 = Thread(target=producer, args=(q,))
# t1.run()
# t2.run()

## 给关键部分加锁

In [4]:
import threading

class SharedCounter:
    def __init__(self, initial_value = 0):
        self._value = initial_value
        self._value_lock = threading.Lock()

    def inc(self, delta=1):
        with self._value_lock:
            self._value += delta

    def decr(self, delta=1):
        with self._value_lock:
            self._value -= delta

## 防止死锁的加锁机制

In [8]:
import threading
from contextlib import contextmanager

_local = threading.local()

@contextmanager
def acquire(*locks):
    locks = sorted(locks, key=lambda x : id(x))
    acquired = getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock Order Violation')
    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [9]:
import threading
x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():
    while True:
        with acquire(x_lock, y_lock):
            print('Thread-1')

def thread_2():
    while True:
        with acquire(x_lock, y_lock):
            print('Thread-2')

## 保存线程的状态信息

In [10]:
from socket import socket, AF_INET, SOCK_STREAM
import threading

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.local = threading.local()

    def __enter__(self):
        if hasattr(self.local, 'sock'):
            raise RuntimeError('Already connected')
        self.local.sock = socket(self.family, self.type)
        self.local.sock.connect(self.address)
        return self.local.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.local.sock.close()
        del self.local.sock

In [13]:
from functools import partial

def test(conn):
    with conn as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv, 8192), b''))
    print('Got {} bytes'.format(len(resp)))

conn = LazyConnection(('www.python.org', 80))
t1 = threading.Thread(target=test, args=(conn,))
t2 = threading.Thread(target=test, args=(conn,))
t1.start()
t2.start()
t1.join()
t2.join()

Got 392 bytes
Got 392 bytes


## 创建一个线程池

In [14]:
from concurrent.futures import ThreadPoolExecutor
import urllib.request

def fetch_url(url):
    u = urllib.request.urlopen(url)
    data = u.read()
    return data

pool = ThreadPoolExecutor(10)

a = pool.submit(fetch_url, 'http://www.python.org')
b = pool.submit(fetch_url, 'http://www.pypy.org')

print('Got {} bytes'.format(len(a.result())))
print('Got {} bytes'.format(len(b.result())))

Got 48808 bytes
Got 6836 bytes


后面的章节略